<a href="https://colab.research.google.com/github/ShamamaAslam/PDC-Assignments/blob/main/PDC_Assignment05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get update -qq
!sudo apt-get install -y mpich


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37
Suggested packages:
  mpich-doc
The following NEW packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37 mpich
0 upgraded, 5 newly installed, 0 to remove and 39 not upgraded.
Need to get 14.2 MB of archives.
After this operation, 102 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libslurm37 amd64 21.08.5-2ubuntu1 [542 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 hwloc-nox amd64 2.7.0-2ubuntu1 [205 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich12 amd64 4.0-3 [5,866 kB]
Get:4 http://

In [6]:
%%bash
cat > task1.c <<'EOF'
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc,char *argv[]){
    MPI_Init(&argc,&argv);
    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD,&rank);

    int N=10;
    int *A=NULL;
    if(rank==0){
        A=(int*)malloc(N*sizeof(int));
        for(int i=0;i<N;i++)A[i]=i+1;
        printf("Vector A: ");
        for(int i=0;i<N;i++)printf("%d ",A[i]);
        printf("\n");
    }
    MPI_Finalize();
    return 0;
}
EOF

mpicc task1.c -o task1
mpirun --allow-run-as-root --oversubscribe -np 4 ./task1



Vector A: 1 2 3 4 5 6 7 8 9 10 


In [7]:
%%bash
cat > task2.c <<'EOF'
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc,char *argv[]){
 MPI_Init(&argc,&argv);
 int rank,size;
 MPI_Comm_rank(MPI_COMM_WORLD,&rank);
 MPI_Comm_size(MPI_COMM_WORLD,&size);

 int N=12;
 int each=N/size;
 int *A=NULL;
 if(rank==0){
   A=(int*)malloc(N*sizeof(int));
   for(int i=0;i<N;i++)A[i]=i+1;
 }

 int *part=(int*)malloc(each*sizeof(int));
 MPI_Scatter(A,each,MPI_INT,part,each,MPI_INT,0,MPI_COMM_WORLD);

 printf("Process %d received:",rank);
 for(int i=0;i<each;i++)printf(" %d",part[i]);
 printf("\n");

 MPI_Finalize();
 return 0;
}
EOF

mpicc task2.c -o task2
mpirun --allow-run-as-root --oversubscribe -np 4 ./task2


Process 0 received: 1 2 3
Process 2 received: 7 8 9
Process 1 received: 4 5 6
Process 3 received: 10 11 12


In [8]:
%%bash
cat > task3.c <<'EOF'
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc,char *argv[]){
 MPI_Init(&argc,&argv);
 int rank,size;
 MPI_Comm_rank(MPI_COMM_WORLD,&rank);
 MPI_Comm_size(MPI_COMM_WORLD,&size);

 int N=12;
 int each=N/size;
 int *A=NULL;
 if(rank==0){
   A=(int*)malloc(N*sizeof(int));
   for(int i=0;i<N;i++)A[i]=i+1;
 }

 int *part=(int*)malloc(each*sizeof(int));
 MPI_Scatter(A,each,MPI_INT,part,each,MPI_INT,0,MPI_COMM_WORLD);

 int local_sum=0;
 for(int i=0;i<each;i++)local_sum+=part[i];

 printf("Process %d local sum=%d\n",rank,local_sum);

 MPI_Finalize();
 return 0;
}
EOF

mpicc task3.c -o task3
mpirun --allow-run-as-root --oversubscribe -np 4 ./task3


Process 0 local sum=6
Process 1 local sum=15
Process 2 local sum=24
Process 3 local sum=33


In [9]:
%%bash
cat > task4.c <<'EOF'
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc,char *argv[]){
 MPI_Init(&argc,&argv);
 int rank,size;
 MPI_Comm_rank(MPI_COMM_WORLD,&rank);
 MPI_Comm_size(MPI_COMM_WORLD,&size);

 int N=12;
 int each=N/size;
 int *A=NULL;
 if(rank==0){
   A=(int*)malloc(N*sizeof(int));
   for(int i=0;i<N;i++)A[i]=i+1;
 }

 int *part=(int*)malloc(each*sizeof(int));
 MPI_Scatter(A,each,MPI_INT,part,each,MPI_INT,0,MPI_COMM_WORLD);

 int local_sum=0;
 for(int i=0;i<each;i++)local_sum+=part[i];

 int total_sum=0;
 MPI_Reduce(&local_sum,&total_sum,1,MPI_INT,MPI_SUM,0,MPI_COMM_WORLD);

 if(rank==0)printf("Total Sum (MPI_Reduce)=%d\n",total_sum);

 MPI_Finalize();
 return 0;
}
EOF

mpicc task4.c -o task4
mpirun --allow-run-as-root --oversubscribe -np 4 ./task4


Total Sum (MPI_Reduce)=78


In [10]:
%%bash
cat > task5.c <<'EOF'
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc,char *argv[]){
 MPI_Init(&argc,&argv);
 int rank,size;
 MPI_Comm_rank(MPI_COMM_WORLD,&rank);
 MPI_Comm_size(MPI_COMM_WORLD,&size);

 int N=12;
 int each=N/size;
 int *A=NULL;
 if(rank==0){
   A=(int*)malloc(N*sizeof(int));
   for(int i=0;i<N;i++)A[i]=i+1;
 }

 int *part=(int*)malloc(each*sizeof(int));
 MPI_Scatter(A,each,MPI_INT,part,each,MPI_INT,0,MPI_COMM_WORLD);

 int local_sum=0;
 for(int i=0;i<each;i++)local_sum+=part[i];

 int total_sum=0;
 MPI_Reduce(&local_sum,&total_sum,1,MPI_INT,MPI_SUM,0,MPI_COMM_WORLD);

 if(rank==0){
   int expected=N*(N+1)/2;
   printf("Total Sum (MPI)=%d\nExpected Sum=%d\nDifference=%d\n",
          total_sum,expected,expected-total_sum);
 }
 MPI_Finalize();
 return 0;
}
EOF

mpicc task5.c -o task5
mpirun --allow-run-as-root --oversubscribe -np 4 ./task5


Total Sum (MPI)=78
Expected Sum=78
Difference=0


In [11]:
%%bash
cat > bonus.c <<'EOF'
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc,char *argv[]){
 MPI_Init(&argc,&argv);
 int rank,size;
 MPI_Comm_rank(MPI_COMM_WORLD,&rank);
 MPI_Comm_size(MPI_COMM_WORLD,&size);

 int N=10;  // not divisible by number of processes
 int base=N/size;
 int rem=N%size;
 int local_n=base+(rank<rem?1:0);

 int *A=NULL,*sendcounts=NULL,*displs=NULL;
 if(rank==0){
   A=(int*)malloc(N*sizeof(int));
   sendcounts=(int*)malloc(size*sizeof(int));
   displs=(int*)malloc(size*sizeof(int));
   for(int i=0;i<N;i++)A[i]=i+1;
   int offset=0;
   for(int i=0;i<size;i++){
     sendcounts[i]=base+(i<rem?1:0);
     displs[i]=offset;
     offset+=sendcounts[i];
   }
 }

 int *part=(int*)malloc(local_n*sizeof(int));
 MPI_Scatterv(A,sendcounts,displs,MPI_INT,part,local_n,MPI_INT,0,MPI_COMM_WORLD);

 int local_sum=0;
 for(int i=0;i<local_n;i++)local_sum+=part[i];

 int total_sum=0;
 MPI_Reduce(&local_sum,&total_sum,1,MPI_INT,MPI_SUM,0,MPI_COMM_WORLD);

 if(rank==0)printf("Total Sum (Uneven N)=%d\n",total_sum);

 MPI_Finalize();
 return 0;
}
EOF

mpicc bonus.c -o bonus
mpirun --allow-run-as-root --oversubscribe -np 4 ./bonus


Total Sum (Uneven N)=55
